In [1]:
# from langchain_community.llms import HuggingFaceEndpoint
# the new way
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate,FewShotPromptTemplate

In [2]:
from dotenv import load_dotenv
import os

# Provide the filename as a string
load_dotenv('../.env')

True

In [ ]:
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

model_kwargs = {
    "max_new_tokens": 1000, # Maximum tokens to generate
    "max_length": 5000, # Maximum length of input + output
    "temperature": 0.1, # Controls randomness of output
    "timeout": 6000,
    # "task":'conversational',
}


llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN,
    # you specify the task or not
    # You can also specify the task in the model_kwargs or within here
    # task = 'conversational',
    **model_kwargs,
)

## **Chain Prompting**

Chain Prompting refers to the practice of chaining consecutive prompts, where the output of a previous prompt becomes the input of the successive prompt.

To use chain prompting with LangChain, you could:

- Extract relevant information from the generated response.
- Use the extracted information to create a new prompt that builds upon the previous response.
- Repeat steps as needed until the desired output is achieved.

In [4]:
# Question template
prompt_template_question = """
You are an expert journalist that has 
written countless articles and journals for major newspaper and blog publishers.
Kindly provide ONLY the name(s) of the person(s) started {company} ? and nothing else.
ONLY PROVIDE THE NAMES
Answer:
"""

prompt_question_input_variables = ['company']

prompt_question = PromptTemplate(
    template=prompt_template_question,
    input_variables=prompt_question_input_variables
)


# Fact template
prompt_template_fact = """
Provide a brief article of how the {person} started the company. Provide a title for the article
Answer:"""

prompt_fact_input_variables = ['person']
prompt_fact = PromptTemplate(
    template=prompt_template_fact,
    input_variables=prompt_fact_input_variables
)


In [5]:
user_input = input("Who founded > ")

input_data = {"company": user_input}

llm_chain_question = prompt_question | llm

response_question = llm_chain_question.invoke(input_data)

# Extract the person's name from the response
person = response_question.strip()
print(person) # checking intermediate results

Bill Gates and Paul Allen


In [6]:
# Input data for the second prompt
fact_input_data = {"person":person}

llm_chain_fact = prompt_fact | llm

response_fact = llm_chain_fact.invoke(fact_input_data)

In [7]:
print(f"Person Name(s): {person}\n")
print(f"Facts:\n{response_fact.strip()}")

Person Name(s): Bill Gates and Paul Allen

Facts:
Title: "The Genesis of a Tech Titan: The Unlikely Partnership of Bill Gates and Paul Allen that Launched Microsoft"

In the annals of tech history, few partnerships have left an indelible mark like that of Bill Gates and Paul Allen. The duo, who met in their teenage years, would go on to create one of the most influential companies in the world: Microsoft.

Their story began in 1970, at the prestigious Lakeside School in Seattle, Washington. Both Gates and Allen were computer enthusiasts, spending countless hours in the school's computer lab. It was here that they first crossed paths, their shared passion for computers forming the foundation of their future collaboration.

After graduating from college, Gates moved to New Mexico to work for Honeywell, while Allen went to work for Boeing. However, their shared dream of creating software for personal computers was too strong to ignore. In 1975, they decided to leave their respective jobs 